## RDBMS data modeling

In [1]:
import psycopg2
from sql_queries import user_table_insert, fn_agg, ln_agg
import requests

In [2]:
try: 
    conn = psycopg2.connect("host=127.0.0.1 dbname=zylotechdb user=student password=student")
    cur = conn.cursor()
except psycopg2.Error as e: 
    print("Error: Could not make connection to the Postgres database")
    print(e)

In [3]:
# api-endpoint 
URL = "https://reqres.in/api/users?page="
#data requesting from endpoint pages
pages = [1, 2, 3, 4]

In [4]:
for page in pages:
    URL = "https://reqres.in/api/users?page="+str(page)
    print(URL)
    try:
        res = requests.get(url = URL)
        res_json = res.json()
    except requests.exceptions.HTTPError as e:
        print(e)
        
    if res_json['data']:
        users = res_json['data']
        for user in users:
            id1 = user['id']
            email = user['email']
            first_name = user['first_name']
            last_name = user['last_name']
            avatar = user['avatar']
#                 print(id1, email, first_name, last_name, avatar)
            values = (id1,  first_name, last_name,email, avatar, "now()")
    
            try:
                cur.execute(user_table_insert, values)
            except psycopg2.Error as e: 
                print("Error: Inserting Rows")
                print (e) 
    else:
        print("No users list in the API end point.")

https://reqres.in/api/users?page=1
https://reqres.in/api/users?page=2
https://reqres.in/api/users?page=3
https://reqres.in/api/users?page=4


In [5]:
try: 
    cur.execute("SELECT * FROM users;")
except psycopg2.Error as e: 
    print("Error: select *")
    print (e)

row = cur.fetchone()
while row:
    #print first_name and last_name
    print(row[2], row[3])
    row = cur.fetchone()

George Bluth
Janet Weaver
Emma Wong
Eve Holt
Charles Morris
Tracey Ramos
Michael Lawson
Lindsay Ferguson
Tobias Funke
Byron Fields
George Edwards
Rachel Howell


In [6]:
# Count of last name starting with same letter:
def aggregation_metrics(query):
    try: 
        cur.execute(query)
        rows = cur.fetchall()
        return rows
    except psycopg2.Error as e: 
        print("Error: ", query)
        print (e)

In [7]:
fn_agg_res = aggregation_metrics(fn_agg)
ln_agg_res = aggregation_metrics(ln_agg)

In [8]:
for row in fn_agg_res:
    print(row)   

('B', 1)
('C', 1)
('E', 2)
('G', 2)
('J', 1)
('L', 1)
('M', 1)
('R', 1)
('T', 2)


In [9]:
for row in ln_agg_res:
    print(row)

('B', 1)
('E', 1)
('F', 3)
('H', 2)
('L', 1)
('M', 1)
('R', 1)
('W', 2)


## NoSQL data modeling

In [10]:
import cassandra
from cassandra.cluster import Cluster
from sql_queries import ac_users_table_create, ac_users_table_insert

In [11]:
try: 
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)

In [12]:
try:
    session.set_keyspace('zylotechdb2')
except Exception as e:
    print(e)

In [13]:
try:
    session.execute(ac_users_table_create)
except Exception as e:
    print(e)

In [14]:
def insert_data(session, query):
    for page in pages:
        URL = "https://reqres.in/api/users?page="+str(page)
        print(URL)
        try:
            res = requests.get(url = URL)
            res_json = res.json()
        except requests.exceptions.HTTPError as e:
            print(e)

        if res_json['data']:
            users = res_json['data']
            for user in users:
                id1 = user['id']
                email = user['email']
                first_name = user['first_name']
                last_name = user['last_name']
                avatar = user['avatar']
                session.execute(query, (int(id1), str(email), str(first_name), str(last_name), str(avatar)))

In [15]:

# INSERT into the table
insert_data(session, ac_users_table_insert)

https://reqres.in/api/users?page=1
https://reqres.in/api/users?page=2
https://reqres.in/api/users?page=3
https://reqres.in/api/users?page=4
